<a href="https://colab.research.google.com/github/Eddiebee/AI-Craft/blob/main/Prompt_tracking_with_Comet_and_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logging, Tracking, and Debugging Prompts using Comet

In this section, we will demonstrate how to log, track, and debug prompt using the `comet-llm` library. `comet-llm` is an open-sourced repo managed by Comet. Please give the repo star if you have a chance and submit any feedback you have! https://github.com/comet-ml/comet-llm

Let's first load all the necessary libraries:


In [2]:
! pip install cohere comet-llm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00


In [5]:
import cohere
import os
import IPython
import json
import pandas as pd
import numpy as np
import comet_llm
import urllib


The function below helps to generate the final results from the model after calling the Cohere _chat_ API:

In [3]:
from google.colab import userdata
COHERE_API_KEY = userdata.get("COHERE_TRIAL_KEY")

In [6]:
co = cohere.Client(COHERE_API_KEY)

In [28]:
def get_completion(message,
                   model="command-r-plus",
                   temperature=0,
                   max_tokens=300):
    response = co.chat(
        model=model,
        message=message,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.text

### Load the Data

The code below loads both the few-shot demonstrations and the validation dataset used for testing the model.

In [9]:
# print markdown
def print_markdown(text):
    """Prints text as markdown"""
    IPython.display.display(IPython.display.Markdown(text))

# load validation data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/article-tags.json")
val_data = json.load(f)

# load few shot data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/few_shot.json")
few_shot_data = json.load(f)

The following is a helper function to obtain the final predictions from the model given a prompt template (e.g., zero-shot or few-shot) and the provided input data.

In [27]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        preamble = prompt_template.format(input=inputs[i])
        message = preamble
        response = get_completion(message)
        responses.append(response)

    return responses

### Few-Shot

First, we define a few-shot template which will leverage the few-shot demonstration data loaded previously.

In [11]:
# function to define the few-shot template
def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n\n" + "\n".join([ "Abstract: "+ ex["abstract"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# function to sample few shot data
def random_sample_data (data, n):
    return np.random.choice(few_shot_data, n, replace=False)


# the few-shot prefix and suffix
few_shot_prefix = """Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]"""
few_shot_suffix = """Abstract: {input}\nTags:"""

# load 3 samples from few shot data
few_shot_template = get_few_shot_template(few_shot_prefix, few_shot_suffix, random_sample_data(few_shot_data, 3))

In [12]:
few_shot_template

'Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don\'t find model names in the abstract or you are not sure, return ["NA"]\n\nAbstract: Topological deep learning is a rapidly growing field that pertains to the development of deep learning models for data supported on topological domains such as simplicial complexes, cell complexes, and hypergraphs, which generalize many domains encountered in scientific computations. In this paper, we present a unifying deep learning framework built upon a richer data structure that includes widely adopted topological domains. Specifically, we first introduce combinatorial complexes, a novel type of topological domain. Combinatorial complexes can be seen as generalizations of graphs that maintain certain desirable properties. Similar to hypergraphs, combinatorial complexes impose no constraints on the set of relations. In addition, combinato

### Zero-Shot Template

The code below defines the zero-shot template. Note that we use the same instruction from the few-shot prompt template. But in this case, we don't use the demonstrations.

In [13]:
zero_shot_template = """
Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]

Abstract: {input}
Tags:
"""

### Get Predictions

We then generated all the predictions using the validation data as inputs:

In [30]:
# get the predictions

abstracts = [val_data[i]["abstract"] for i in range(len(val_data))]
few_shot_predictions = get_predictions(few_shot_template, abstracts)
zero_shot_predictions = get_predictions(zero_shot_template, abstracts)
expected_tags = [str(val_data[i]["tags"]) for i in range(len(val_data))]

In [31]:
print("Few shot predictions")
print(few_shot_predictions)
print("\n\nZero shot predictions")
print(zero_shot_predictions)
print("\n\nExpected tags")
print(expected_tags)

Few shot predictions
['["CCNNs"]\n["SadTalker", "ExpNet", "PoseVAE"]\n["ChatGPT", "GPT-4", "LLaMA", "Alpaca"]\n["Evol-Instruct", "WizardLM", "ChatGPT"]', '["CCNNs", "SadTalker", "ExpNet", "PoseVAE", "ChatGPT", "GPT-4", "LLaMA", "Alpaca", "FLAN-T5"]', '["NA"]', '["NA"]', '["CCNNs"]\n["SadTalker", "ExpNet", "PoseVAE"]\n["ChatGPT", "GPT-4", "LLaMA", "Alpaca"]\n["ChatGPT"]', '["ViT"]', '["SadTalker", "ExpNet", "PoseVAE", "ChatGPT", "GPT-4", "LLaMA", "Alpaca", "Inpaint Anything (IA)"]', '["Anything-3D", "BLIP", "Segment-Anything"]', '["CCNNs"]\n["SadTalker", "ExpNet", "PoseVAE"]\n["ChatGPT", "GPT-4", "LLaMA", "Alpaca"]\n["Chameleon"]', '["NA"]']


Zero shot predictions
['["WizardLM", "LLaMA", "ChatGPT"]', '["FLAN-T5"]', '["NA"]', '["PAXQA"]', '["ChatGPT"]', '["ViT"]', '["Segment-Anything Model", "Inpaint Anything"]', '["Anything-3D", "BLIP", "Segment-Anything"]', '["Chameleon"]', '["NA"]']


Expected tags
["['LLaMA', 'ChatGPT', 'WizardLM']", "['FLAN-T5', 'FLAN']", "['NA']", "['PAXQA']", "['

### Log Prompt Results

Finally, we log the prompt + results to Comet. Note that we are logging both the few-shot and zero-shot results, together with all the metadata and tags.

In [35]:
# log the predictions in Comet along with the ground truth for comparison

# set up comet
COMET_API_KEY = userdata.get("COMET_API_KEY")
COMET_WORKSPACE = "eddiebee"

# initialize comet
comet_llm.init(COMET_API_KEY,
               COMET_WORKSPACE,
               project="ml-paper-tagger-prompts")

# log the predictions
for i in range(len(expected_tags)):
    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=few_shot_template.format(input=abstracts[i]),
        prompt_template=few_shot_template,
        output=few_shot_predictions[i],
        tags = ["cohere", "command-r-plus" "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

    # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=abstracts[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["cohere", "command-r-plus" "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


Prompt logged to https://www.comet.com/eddiebee/ml-paper-tagger-prompts


INFO:comet_llm.summary:Prompt logged to https://www.comet.com/eddiebee/ml-paper-tagger-prompts
